### Libraries

In [1]:
from typing import Union
import numpy as np
from qclib.gates.mcu import MCU
from qiskit import (QuantumCircuit,
                    QuantumRegister,
                    transpile)
from qiskit.providers.fake_provider import GenericBackendV2
# from joblib import Parallel, delayed
import matplotlib.pyplot as plt

In [2]:
class Utilities:

    def __init__(self, operator, error):
        self.operator = operator
        self.error = error

    @staticmethod
    def pauli_matrices(string: str) -> Union[np.ndarray, None]:
        string_case = string.lower()
        match string_case:
            case "x":
                return np.array([
                    [0, 1],
                    [1, 0]
                ])
            case "y":
                return np.array([
                    [0, -1j],
                    [1j, 0]
                ])
            case "z":
                return np.array([
                    [1, 0],
                    [0, -1]
                ])
            case _:
                return None

    @staticmethod
    def transpose_conjugate(operator):
        return np.conjugate(np.transpose(operator))

    @staticmethod
    def pyramid_size(operator, error):
        mcu_dummy = MCU(operator, num_controls=100, error=error)
        # will be changed by mcu_dummy._get_n_base(x_dagger, error)
        return mcu_dummy._get_num_base_ctrl_qubits(operator, error)


In [13]:
def built_circuit(pauli_string='x', error=0.1, approximated = True):
    
    pauli_matrix = Utilities.pauli_matrices(pauli_string)
    pauli_matrix_dagger = Utilities.transpose_conjugate(pauli_matrix)
    n_base = Utilities.pyramid_size(pauli_matrix_dagger, error)

    controls = QuantumRegister(n_base, 'controls')
    target = QuantumRegister(1, 'target')
    # classical = ClassicalRegister(1, "classic")

    circ = QuantumCircuit(controls, target) #, classical)
    for i in range(len(controls)):
        circ.x(i)
    circ.x(target)
    if approximated:
      MCU.mcu(circ, pauli_matrix_dagger, controls, target, error)
    else:
      MCU.mcu(circ, pauli_matrix_dagger, controls, target, 0)
    # circ.measure(target, [0])
    circ.measure_all()
    return circ

In [15]:
approx_circuit = built_circuit('x', 0.1)
print(approx_circuit)

            ┌───┐┌──────────────┐ ░ ┌─┐                  
controls_0: ┤ X ├┤0             ├─░─┤M├──────────────────
            ├───┤│              │ ░ └╥┘┌─┐               
controls_1: ┤ X ├┤1             ├─░──╫─┤M├───────────────
            ├───┤│              │ ░  ║ └╥┘┌─┐            
controls_2: ┤ X ├┤2             ├─░──╫──╫─┤M├────────────
            ├───┤│              │ ░  ║  ║ └╥┘┌─┐         
controls_3: ┤ X ├┤3 Ldmcuapprox ├─░──╫──╫──╫─┤M├─────────
            ├───┤│              │ ░  ║  ║  ║ └╥┘┌─┐      
controls_4: ┤ X ├┤4             ├─░──╫──╫──╫──╫─┤M├──────
            ├───┤│              │ ░  ║  ║  ║  ║ └╥┘┌─┐   
controls_5: ┤ X ├┤5             ├─░──╫──╫──╫──╫──╫─┤M├───
            ├───┤│              │ ░  ║  ║  ║  ║  ║ └╥┘┌─┐
    target: ┤ X ├┤6             ├─░──╫──╫──╫──╫──╫──╫─┤M├
            └───┘└──────────────┘ ░  ║  ║  ║  ║  ║  ║ └╥┘
    meas: 7/═════════════════════════╩══╩══╩══╩══╩══╩══╩═
                                     0  1  2  3  4  5  6 


In [16]:
real_circuit = built_circuit('x', 0, False)
print(real_circuit)

OverflowError: cannot convert float infinity to integer

In [98]:
def hist(circuit):
    size = len(circuit)
    # Generate a size-qubit simulated backend
    backend = GenericBackendV2(num_qubits=size)
    
    # Transpile the circuit to a circuit that can be directly executed by the backend
    transpiled_circuit = transpile(circuit, backend)
    transpiled_circuit.save_statevector()
    
    # Run the transpiled circuit using the simulated fake backend
    job = backend.run(transpiled_circuit)
    result = job.result()
    output_state = np.real(result.get_statevector(transpiled_circuit, decimals=100))
    return output_state

In [99]:
rc_state_vector = hist(real_circuit)
ac_state_vector = hist(approx_circuit)

### Adicionar medida de fidelidade.

In [100]:
from qiskit.quantum_info import Statevector, DensityMatrix, state_fidelity

In [101]:
sv1 = Statevector(rc_state_vector)
sv2 = Statevector(ac_state_vector)
fidelity = state_fidelity(sv1, sv2)
fidelity

QiskitError: 'Input quantum state is not a valid'

In [96]:
c1 = QuantumCircuit(3)
c1.x(range(3))
c1.ccx(0,1,2)
c1.measure_all()
c1.draw('text')

┌───┐      ░ ┌─┐      
   q_0: ┤ X ├──■───░─┤M├──────
        ├───┤  │   ░ └╥┘┌─┐   
   q_1: ┤ X ├──■───░──╫─┤M├───
        ├───┤┌─┴─┐ ░  ║ └╥┘┌─┐
   q_2: ┤ X ├┤ X ├─░──╫──╫─┤M├
        └───┘└───┘ ░  ║  ║ └╥┘
meas: 3/══════════════╩══╩══╩═
                      0  1  2

In [97]:
sv3 = hist(c1)
print(len(sv3))

256
